In [1]:
import pandas as pd
#import numpy as np
#import glob
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss

# Load Data

In [3]:
#data_dir = '../input/lish-moa'
data_dir = '../data/01_raw'

sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
targets_scored = train_targets_scored.columns[1:]

# Launch data pipelines in parallel

In [5]:
#launch models
import subprocess
import time
import random

lsub = []

# mdl1 = subprocess.Popen("python data_v1.py".split())
# lsub += [mdl1]
mdl1b = subprocess.Popen("python data_v3.py".split())
lsub += [mdl1b]
# mdl2 = subprocess.Popen("python data_v2.py".split())
# lsub += [mdl2]
# mdl3 = subprocess.Popen("python data_v3.py".split())
# lsub += [mdl3]

while len(lsub) > 0:
    random.shuffle(lsub)
    it = iter(lsub)
    p = next(it)

    print(p)
    if p.poll() != None:
        print(f"{p} is done")
        lsub.remove(p)
    time.sleep(10)

<subprocess.Popen object at 0x7feb6260e9d0> is done
<subprocess.Popen object at 0x7feb6264a890> is done


# Launch models in parallel

In [6]:
#All models are based on 5folds - 3 seeds
#model1 - DNN on scored targets only
#model2 - DNN on scored targets and unscored targets
#model3 - Resnet using scored targets and 
#model4 - tabnet model

#launch models
import subprocess
import time
import random

lsub = []

mdl1 = subprocess.Popen("python mdl2_3.py".split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
lsub += [mdl1]
mdl2 = subprocess.Popen("python mdl3_3.py".split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
lsub += [mdl2]
mdl3 = subprocess.Popen("python mdl1.py".split(), stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
lsub += [mdl3]




while len(lsub) > 0:
    random.shuffle(lsub)
    it = iter(lsub)
    p = next(it)

    # print(p.communicate()[0])
    print(p)
    if p.poll() != None:
        print(f"{p} is done")
        lsub.remove(p)
    time.sleep(10)
    

<subprocess.Popen object at 0x7feb6260b9d0> is done
<subprocess.Popen object at 0x7feb6260e990> is done


# Optimize ratios

In [53]:
oof_v1 = pd.read_csv('./oof_v1.csv.gz',compression="gzip")
oof_v2 = pd.read_csv('./oof_v2_3.csv.gz',compression="gzip")
oof_v3 = pd.read_csv('./oof_v3_3.csv.gz',compression="gzip")

In [54]:
#Optimize ratios v1,v2...
#ratios = [0.25,0.4,0.35]
ratios = [0.35,0.4,0.25]
#ratios = [.0,.0,1]

train = oof_v2.copy()
train[targets_scored] =  ratios[0]*oof_v1[targets_scored] \
                         + ratios[1]*oof_v2[targets_scored] \
                         + ratios[2]*oof_v3[targets_scored]


In [55]:
y_true = train_targets_scored.iloc[:,1:].values
y_pred = train[targets_scored].values

score = 0
for i in range(len(targets_scored)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / len(targets_scored)
    
print("CV log_loss: ", score)
print(roc_auc_score(y_true,y_pred))

CV log_loss:  0.015837040431665825
0.6556263804276584


In [ ]:
#0.35,0.4,0.25 - CV log_loss:  0.015837040431665825 - 0.6556263804276584

In [56]:
sub_v1 = pd.read_csv('./sub_v1.csv.gz')
sub_v2 = pd.read_csv('./sub_v2_3.csv.gz')
sub_v3 = pd.read_csv('./sub_v3_3.csv.gz')

In [57]:
test = sub_v2.copy()
test[targets_scored] =  ratios[0]*sub_v2[targets_scored] \
                            + ratios[1]*sub_v3[targets_scored]

In [58]:
#Prepare submission
submission = pd.read_csv(data_dir+'/sample_submission.csv')

#Add blended ratios
submission.iloc[:, 1:] = test[targets_scored]
submission.to_csv('submission.csv', index=False)